In [8]:
import os
from pathlib import Path

PROJECT_DIR  = '/content'
os.chdir(Path(PROJECT_DIR))
print("Current Working Directory:", Path.cwd())

Current Working Directory: /content


In [12]:
!if [ -d "pdm" ]; then rm -rf pdm; fi
!git clone https://github.com/soroushdty/pdm.git
REPO_DIR = Path.cwd() / "pdm"
print(REPO_DIR)
os.chdir(REPO_DIR)
print("Current Working Directory:", Path.cwd())

Cloning into 'pdm'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 150 (delta 75), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (150/150), 73.58 KiB | 5.25 MiB/s, done.
Resolving deltas: 100% (75/75), done.
/content/pdm/pdm
Current Working Directory: /content/pdm/pdm


In [10]:
from funcs.requirements_utils import install_missing
installed = install_missing(Path(PROJECT_DIR) / "requirements.txt", quiet=False)
print("Installed:", installed)

FileNotFoundError: requirements file not found: /content/requirements.txt

In [4]:
# Load config
from funcs.config_loader import load_config, validate_config
cfg = validate_config(load_config(CONFIG_PATH))
cfg

{'DIR_INPUT': './input',
 'output_path': './output',
 'patient_col': 'Patient',
 'physician_col': 'Physician',
 'item_col': 'Item',
 'classes': ['Behavioral health',
  'Diagnoses',
  'Disabilities',
  'Infectious diseases',
  'Genetics',
  'Medications',
  'Sexual and reproductive health',
  'Social determinants of health',
  'Violence',
  'Other'],
 'other_class': ['Disabilities',
  'Infectious diseases',
  'Genetics',
  'Sexual and reproductive health',
  'Violence',
  'Social determinants of health'],
 'llms': ['sentence-transformers/all-MiniLM-L6-v2',
  'sentence-transformers/all-mpnet-base-v2',
  'sentence-transformers/multi-qa-MiniLM-L6-cos-v1',
  'pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb',
  'sentence-transformers/biomed-mpnet-base',
  'cambridgeltl/SapBERT-from-PubMedBERT-fulltext',
  'bert-base-uncased',
  'roberta-base',
  'distilbert-base-uncased',
  'nreimers/MiniLM-L6-H384-uncased',
  'emilyalsentzer/Bio_ClinicalBERT',
  'monologg/biobert_v1.1_pubmed',
  'bi

In [ ]:
import os
from pathlib import Path
backend = 'colab' # Switch backend
PROJECT_DIR = backend_select(backend)
os.chdir(PROJECT_DIR)
print("Current Working Directory:", Path.cwd())



Current Working Directory: /content/pdm
Installed: []


In [ ]:
# Load config
from funcs.config_loader import load_config, validate_config

CONFIG_PATH = os.path.join(PROJECT_DIR, "config.json")
cfg = validate_config(load_config(CONFIG_PATH))
cfg

ValueError: Config is missing required key: 'dataset' (path to xlsx).

In [ ]:
# Create run directory + logger, then run pipeline
from funcs.paths import make_run_dir
from funcs.logger_setup import setup_logger
from funcs.preprocessing import run
from pathlib import Path

run_dir = make_run_dir(cfg["output_path"])
logger = setup_logger(run_dir / "log.txt")

# IMPORTANT: pipeline writes outputs into cfg["output_path"].
# We'll point cfg output_path at this run directory for this run.
#note: resolve later
cfg_run = dict(cfg)
cfg_run["output_path"] = str(run_dir)

[18:56:22 02-01-26] INFO: Logging initialized at: output/18-56_FEB-01/log.txt


In [ ]:
p = run(cfg_run)
train = p['paths']['train_csv']
test = p['paths']['test_csv']
train_map = p['paths']['train_map']
test_map = p['paths']['test_map']

FileNotFoundError: Dataset file not found at: input/dataset.xlsx